In [1]:
import numpy as np  # Für numerische Berechnungen
import pandas as pd  # Für die Datenverarbeitung und Tabellenmanipulation
from sklearn.cluster import AgglomerativeClustering  # Agglomeratives Clustering-Modell aus Scikit-Learn
from sklearn.metrics import silhouette_score, davies_bouldin_score  # Metriken zur Bewertung der Cluster-Qualität
import joblib  # Zum Speichern und Laden von Modellen
import scipy.cluster.hierarchy as sch  # Für hierarchische Clustering-Methoden
import matplotlib.pyplot as plt  # Zur Visualisierung von Cluster-Dendrogrammen
from scipy.spatial.distance import pdist  # Berechnet paarweise Distanzen zwischen Datenpunkten
from scipy.cluster.hierarchy import fcluster, linkage  # Zum Erstellen und Extrahieren von Cluster-Hierarchien
import os

# Erstelle den Ordner output/plots, wenn er noch nicht existiert
os.makedirs("output/plots", exist_ok=True)

# Vorverarbeitete Daten laden (enthalten BEZ und Koordinaten)
df = pd.read_csv("hierarchisch_encoded_data.csv")

# Neue Spalte für finale Cluster-Zuordnung initialisieren
df["Cluster"] = -1

#---------------------------------
# Dendogramme pro Bezirk erstellen
#--------------------------------
# Dictionary für die besten Clusteranzahlen pro Bezirk
best_clusters = {}

# Erstelle Dendrogramme für jeden Bezirk und bestimme die Clusteranzahl automatisch
for bezirk in sorted(df["BEZ"].unique()):
    df_bezirk = df[df["BEZ"] == bezirk].copy()
    
    # Falls zu wenige Punkte vorhanden sind, überspringen
    if len(df_bezirk) < 10:
        print(f"Bezirk {bezirk}: Zu wenige Punkte für ein Dendrogramm. Überspringe.")
        continue

    # Features für Clustering (ohne Geokoordinaten und BEZ)
    features = df_bezirk.drop(columns=["XGCSWGS84", "YGCSWGS84", "BEZ"])
    
    # Berechnung der Linkage-Matrix
    Z = linkage(pdist(features, metric="euclidean"), method="ward")

    # **Automatische Cluster-Anzahl bestimmen** (Schneiden des Dendrogramms)
    max_dist = 0.7 * np.max(Z[:, 2])  # 70% der max. Höhe als Cutoff-Wert (anpassbar)
    cluster_labels = fcluster(Z, max_dist, criterion="distance")
    num_clusters = len(np.unique(cluster_labels))

    # Speichere die optimale Clusteranzahl im Dictionary
    best_clusters[bezirk] = num_clusters

    # Cluster-Zuordnung in das DataFrame übernehmen
    df.loc[df["BEZ"] == bezirk, "Cluster"] = cluster_labels

    # **Dendrogramm plotten mit Schnittlinie**
    plt.figure(figsize=(12, 6))
    sch.dendrogram(Z, no_labels=True, color_threshold=max_dist)
    plt.axhline(y=max_dist, color='r', linestyle='--', label=f"Cluster Cut ({num_clusters} Cluster)")
    plt.title(f"Dendrogramm für Bezirk {bezirk} (Cluster: {num_clusters})")
    plt.xlabel("Datenpunkte")
    plt.ylabel("Distanz")
    plt.legend()

    # Speichern des Dendrogramms
    plt.savefig(f"output/plots/dendrogram_bezirk_{bezirk}.png", dpi=100, bbox_inches="tight")
    plt.close()

    print(f"Bezirk {bezirk}: {num_clusters} Cluster erkannt.")

# Speichern der Cluster-Ergebnisse
df.to_csv("hierarchisch_final_clusters_pro_bezirk.csv", index=False)

# Speichern des best_clusters-Dictionaries als CSV
best_clusters_df = pd.DataFrame(list(best_clusters.items()), columns=["Bezirk", "Anzahl_Cluster"])
best_clusters_df.to_csv("best_clusters.csv", index=False)

print("Dendrogramme für alle Bezirke gespeichert, Cluster automatisch zugewiesen und best_clusters als CSV gespeichert!")

#--------------------------------
# --- Clustering pro Bezirk ---
#------------------------------
for bezirk in sorted(df["BEZ"].unique()):
    print(f"\nClustering für Bezirk {bezirk}...")
    
    # Daten für den aktuellen Bezirk
    df_bezirk = df[df["BEZ"] == bezirk].copy()
    
    # Falls zu wenige Punkte vorhanden sind, überspringen
    if len(df_bezirk) < 10:
        print(f"  Zu wenige Punkte in Bezirk {bezirk}. Überspringe Clustering.")
        continue
    
    # Features für Clustering (ohne Geokoordinaten und BEZ)
    features = df_bezirk.drop(columns=["XGCSWGS84", "YGCSWGS84", "BEZ"])
    
    # Die optimale Clusteranzahl für den Bezirk verwenden
    if bezirk in best_clusters:
        num_clusters = best_clusters[bezirk]
    else:
        print(f"  Keine optimale Clusteranzahl für Bezirk {bezirk} gefunden. Überspringe Clustering.")
        continue
    
    # Agglomerative Clustering mit der optimalen Anzahl von Clustern
    clustering = AgglomerativeClustering(n_clusters=num_clusters, linkage="ward")
    labels = clustering.fit_predict(features)
    
    # Cluster-Zuordnung für diesen Bezirk
    df.loc[df["BEZ"] == bezirk, "Cluster"] = labels

# Speichern des finalen Clusterdatensatzes
df.to_csv("hierarchisch_final_clusters_pro_bezirk_mit_optimaler_anzahl.csv", index=False)

# Speichern des letzten verwendeten Clustering-Modells
joblib.dump(clustering, "agglomerative_clustering_model_pro_bezirk_mit_optimaler_anzahl.pkl")

# Gruppieren nach BEZ und die Anzahl der eindeutigen Cluster pro Bezirk ermitteln
clusters_per_bezirk = df.groupby("BEZ")["Cluster"].nunique().reset_index(name="Anzahl_Cluster")

print(clusters_per_bezirk)

# --- Aggregationen: Vergleich Gesamtanzahl der Punkte pro Bezirk und Summe der Cluster-Punkte ---
punkte_pro_bezirk = df.groupby("BEZ").size().reset_index(name="Anzahl_Datenpunkte")
print("Gesamtanzahl der Datenpunkte pro Bezirk:")
print(punkte_pro_bezirk)

punkte_pro_cluster = df.groupby(["BEZ", "Cluster"]).size().reset_index(name="Anzahl_Punkte")
print("\nDatenpunkte pro Cluster in jedem Bezirk:")
print(punkte_pro_cluster)

cluster_summe = df.groupby("BEZ")["Cluster"].count().reset_index(name="Summierte_Cluster_Punkte")
print("\nSummierte Anzahl der Cluster-Punkte pro Bezirk:")
print(cluster_summe)

vergleich = pd.merge(punkte_pro_bezirk, cluster_summe, on="BEZ")
vergleich["Differenz"] = vergleich["Anzahl_Datenpunkte"] - vergleich["Summierte_Cluster_Punkte"]
print("\nVergleich: Gesamtpunkte vs. Summierte Cluster-Punkte pro Bezirk:")
print(vergleich)

print("Clustering pro Bezirk abgeschlossen!")

#--------------------------------------------------------------------------
# Berechnung von Silhouette-Score und Davies-Bouldin-Index pro Bezirk
#-----------------------------------------------------------------------
metrics_results = {}
for bez in sorted(df["BEZ"].unique()):
    df_bezirk = df[df["BEZ"] == bez].copy()
    features = df_bezirk.drop(columns=["XGCSWGS84", "YGCSWGS84", "Cluster", "BEZ"])
    labels = df_bezirk["Cluster"]
    
    # Falls im Bezirk nur ein Cluster gebildet wurde, können Metriken nicht berechnet werden
    if labels.nunique() < 2:
        print(f"Bezirk {bez} hat nur einen Cluster. Silhouette und DBI können nicht berechnet werden.")
        continue

    sil = silhouette_score(features, labels, metric="euclidean")
    dbi = davies_bouldin_score(features, labels)
    
    metrics_results[bez] = {"Silhouette": sil, "DaviesBouldin": dbi}
    print(f"Bezirk {bez}: Silhouette Score = {sil:.3f}, Davies-Bouldin Index = {dbi:.3f}")

results_df = pd.DataFrame(metrics_results).T
results_df.to_csv("clustering_metrics_per_bezirk_mit_optimaler_anzahl.csv", index=True)

print("\nZusammenfassung der besten Clusteranzahlen pro Bezirk:")
for bez, clusters in best_clusters.items():
    print(f"Bezirk {bez}: Optimale Clusteranzahl = {clusters}")


Bezirk 1: 4 Cluster erkannt.
Bezirk 2: 3 Cluster erkannt.
Bezirk 3: 3 Cluster erkannt.
Bezirk 4: 6 Cluster erkannt.
Bezirk 5: 4 Cluster erkannt.
Bezirk 6: 5 Cluster erkannt.
Bezirk 7: 5 Cluster erkannt.
Bezirk 8: 5 Cluster erkannt.
Bezirk 9: 3 Cluster erkannt.
Bezirk 10: 3 Cluster erkannt.
Bezirk 11: 4 Cluster erkannt.
Bezirk 12: 4 Cluster erkannt.
Dendrogramme für alle Bezirke gespeichert, Cluster automatisch zugewiesen und best_clusters als CSV gespeichert!

Clustering für Bezirk 1...

Clustering für Bezirk 2...

Clustering für Bezirk 3...

Clustering für Bezirk 4...

Clustering für Bezirk 5...

Clustering für Bezirk 6...

Clustering für Bezirk 7...

Clustering für Bezirk 8...

Clustering für Bezirk 9...

Clustering für Bezirk 10...

Clustering für Bezirk 11...

Clustering für Bezirk 12...
    BEZ  Anzahl_Cluster
0     1               4
1     2               3
2     3               3
3     4               6
4     5               4
5     6               5
6     7               5
7    